## Introduction

In this notebook, we will use LlamaIndex and LLAMA-2 to answer questions based on a youtube Video.



In [4]:
import pandas as pd
import numpy as np

import os
import sys 

from pathlib import Path

### The video is about Chandrayaan-3 and chances of survival on coldest night. 


In [5]:
YOUTUBE_URL="https://www.youtube.com/watch?v=ma9aN2HNDv0" 

### There are two main approaches to reading this youtube video using LLAMA Index
1. Download the youtube video and extract the audio and use https://llamahub.ai/l/file-audio Plugin to transcribe the Audio. This internally uses OpenAI Whishper
2. Directly use https://llamahub.ai/l/youtube_transcript plugin transcript to transcribe the audio. This uses youtube_transcript_api library to transcribe.

In this notebook, we will use the youttube_transcript plugin in LLAMA HUB to read the youtube video



In [1]:
!pip install youtube_transcript_api -q

In [2]:
!pip install llama-hub -q

In [6]:
from llama_hub.youtube_transcript import YoutubeTranscriptReader

loader = YoutubeTranscriptReader()
documents = loader.load_data(ytlinks=[YOUTUBE_URL])

In [7]:
documents

[Document(id_='c32a6483-dd73-40c4-abcd-378bbafe3eae', embedding=None, metadata={'video_id': 'ma9aN2HNDv0'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='da656380d69dcfc43537eeb26dac9e3bca2c2c685424b6f353eac9a035f9e468', text="[Music]\nlanding on the moon is any country's\ndream and India has accomplished that\nwith an astounding Milestone by finally\nLanding the chandrian 3 mission on the\nmoon successfully in one of the untapped\nlocations where ice and water can be\nfound there but nothing comes without\nchallenges and chandron 3's probes are\nfacing a huge one right now they have to\nsurvive the longest and coldest night of\ntheir lives and the problem is that they\ndon't have eaters to keep them warm well\nthat sounds like a challenge and that is\nwhat we are going to talk about in this\nvideo we will tell you everything you\nneed to know about chandron 3's survival\nchallenge during the lunar night and why\nit is important and challenging

### Index the document
We will use Sentence transformer ``sentence-transformers/all-mpnet-base-v2`` to embed the documents. For the Chat, we will use LLAMA-2 q4 quantised model. 

In [22]:
from llama_index import VectorStoreIndex

from langchain.embeddings import HuggingFaceEmbeddings


In [9]:
model_name = "sentence-transformers/all-mpnet-base-v2"
hf = HuggingFaceEmbeddings(
    model_name=model_name
)



/Users/aiswaryaramachandran/miniforge3/envs/py310-whisper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from llama_index import LangchainEmbedding, ServiceContext
from llama_index.llm_predictor import LLMPredictor
from langchain.llms import LlamaCpp

For the LLM,let us load the LLAMA-13B Chat Model with q4 Quantisation.  In MAC,M1 qith q4_0 models - we can use the Metal GPU for inference

In [11]:
llm = LlamaCpp(
    # You can pass in the URL to a GGML model to download it automatically
    #model_url="https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin",
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path="/Users/aiswaryaramachandran/.cache/lm-studio/models/TheBloke/Llama-2-13B-chat-GGUF/llama-2-13b-chat.Q4_0.gguf",
    n_gpu_layers=1,
    n_batch=512,
    temperature=0.1,
    max_tokens=1024,
    top_p=1,
    verbose=True, # Verbose is required 
    f16_kv=True,
    n_ctx=4096,
    use_mlock=True,n_threads=4

)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/aiswaryaramachandran/.cache/lm-studio/models/TheBloke/Llama-2-13B-chat-GGUF/llama-2-13b-chat.Q4_0.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - ten

In [12]:
llm_predictor=LLMPredictor(llm=llm)
embed_model = LangchainEmbedding(hf) 
service_context = ServiceContext.from_defaults(embed_model=embed_model,llm_predictor=llm_predictor)

index = VectorStoreIndex.from_documents(
    documents, 
    service_context=service_context,
)


In [15]:
query_engine=index.as_query_engine()


In [16]:
response=query_engine.query("What is a Lunar Night?")


llama_print_timings:        load time =  5449.22 ms
llama_print_timings:      sample time =   212.78 ms /   118 runs   (    1.80 ms per token,   554.55 tokens per second)
llama_print_timings: prompt eval time = 41056.91 ms /  2234 tokens (   18.38 ms per token,    54.41 tokens per second)
llama_print_timings:        eval time =  6528.19 ms /   117 runs   (   55.80 ms per token,    17.92 tokens per second)
llama_print_timings:       total time = 48292.27 ms


In [17]:
print(response)

 A lunar night refers to the period when the Moon does not receive sunlight due to its rotation on its axis. During this time, the surface temperature of the moon can drop to as low as -253°C (-424°F), with extreme temperature variations of over 680°F (380°C) between day and night. The lunar night lasts for about 14 Earth days, and the South Pole of the moon experiences more extreme temperatures and longer periods of eternal light than the North Pole.


In [19]:
response=query_engine.query("Can you mention the key points regarding Chandrayaan 3 mission?")

Llama.generate: prefix-match hit

llama_print_timings:        load time =  5449.22 ms
llama_print_timings:      sample time =   418.58 ms /   268 runs   (    1.56 ms per token,   640.27 tokens per second)
llama_print_timings: prompt eval time =   756.45 ms /    20 tokens (   37.82 ms per token,    26.44 tokens per second)
llama_print_timings:        eval time = 15102.28 ms /   267 runs   (   56.56 ms per token,    17.68 tokens per second)
llama_print_timings:       total time = 17213.07 ms


In [20]:
print(response)



Based on the provided context information, here are some key points about the Chandrayaan-3 mission:

1. The mission is India's third lunar mission and the first to land near the lunar South Pole.
2. The mission consists of two probes - Vikram (lander) and Pragyan (rover).
3. The probes are designed to study the topography, mineralogy, surface chemistry, and thermal properties of the lunar South Pole region.
4. The mission aims to explore the potential water resources on the Moon.
5. The probes do not have any heaters on board, which makes surviving the lunar night extremely challenging.
6. The chances of the probes waking up after the lunar night depend on several factors beyond human control, such as their inherent durability, location on the surface, communication with the Indian space agency, and more.
7. Previous Moon missions have used heaters to survive the lunar night, but Chandrayaan-3's probes do not have any heaters.
8. The mission has already accomplished its science goal